In [184]:
# load model

import cobra

# model from:
# Caspeta L, Shoaie S, Agren R, et al (2012) Genome-scale metabolic reconstructions of Pichia stipitis and Pichia pastoris and in silico evaluation of their potentials. BMC Syst Biol 6:24. doi: 10.1186/1752-0509-6-24.
# we tweaked some xml (did we?) to avoid warning popping up

pheast = cobra.io.read_sbml_model("./data/iLC915_rewritten.xml")

In [98]:
# run the model/optimizes for cell growth

pheast.optimize()
pheast.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1293,EX_m1293,1,0,0.00%
m1298,EX_m1298,0.7968,0,0.00%
m1300,EX_m1300,0.4628,0,0.00%
m1301,EX_m1301,0.03308,0,0.00%
m1304,EX_m1304,0.00246,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.0803,0,0.00%
m1291,EX_m1291,-1.597,0,0.00%
m1349,EX_m1349,-3.432,0,0.00%


In [79]:
# find glucose reactions (it can be seen that it is the carbon source being taken up at 1 mmol gDW^-1 h^-1)
pheast.metabolites.m1293.reactions

frozenset({<Reaction EX_m1293 at 0x7fb6ab9ebd90>,
           <Reaction r1145 at 0x7fb6aacf1970>})

In [80]:
# exchange reaction in general

pheast.reactions.EX_m1293


Reaction identifier,EX_m1293
Name,EX_m1293
Memory address,0x07fb6ab9ebd90
Stoichiometry,m1293 <=> alpha-D-Glucose_C6H12O6 <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [81]:
# this reaction seems to be the uptake from medium (it is defined in this model as boundary metabolites -> extracellular --> cytosolic
# and we have to define the medium boudary at reaction from boundary to extracellular)

pheast.reactions.r1145


Reaction identifier,r1145
Name,Uptake of alpha-D-Glucose
Memory address,0x07fb6aacf1970
Stoichiometry,m1293 --> m2 alpha-D-Glucose_C6H12O6 --> alpha-D-Glucose_C6H12O6
GPR,
Lower bound,0.0
Upper bound,1.0


In [82]:
# look for methanol

pheast.reactions.query("methanol", "name")

[<Reaction r1158 at 0x7fb6aacf1370>]

In [83]:
# this is methanol uptake

pheast.reactions.r1158

Reaction identifier,r1158
Name,uptake of methanol
Memory address,0x07fb6aacf1370
Stoichiometry,m1297 --> m1219 Methanol_CH4O --> Methanol_CH4O
GPR,
Lower bound,0.0
Upper bound,0.0


In [117]:
pheast.reactions.r1145.bounds = 0, 0  # no glucose 
pheast.reactions.r1158.bounds = 0, 1  # methanol at 1 mmol gDW^-1 h^-1

In [118]:
# it grows on methanol

pheast.optimize()
pheast.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
m1297,EX_m1297,1,0,0.00%
m1298,EX_m1298,1.258,0,0.00%
m1300,EX_m1300,0.06602,0,0.00%
m1301,EX_m1301,0.004719,0,0.00%
m1304,EX_m1304,0.0003508,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
m1287,EX_m1287,-0.01145,0,0.00%
m1291,EX_m1291,-0.372,0,0.00%
m1349,EX_m1349,-1.643,0,0.00%


In [259]:
# no methane yet

pheast.metabolites.query("methane", "name")

[<Metabolite boundary_methane at 0x7fb6a58de220>,
 <Metabolite extracellular_methane at 0x7fb6a58de310>,
 <Metabolite cytosolic_methane at 0x7fb6a6ba7d00>]

In [243]:
# we add methane

b_methane = cobra.Metabolite(
    'boundary_methane',
    formula='C1H4',
    name='bou_methane',
    compartment='C_b')
e_methane = cobra.Metabolite(
    'extracellular_methane',
    formula='C1H4',
    name='ex_methane',
    compartment='C_e')
#c_methane = cobra.Metabolite(
#    'cytosolic_methane',
#    formula='C1H4',
#    name='cyt_methane',
#    compartment='C_c')

pheast.add_metabolites([b_methane,e_methane,c_methane])

In [244]:
# make reactions

EX_methane = cobra.Reaction(
            'EX_methane',
            name = 'Exchange Reaction Methane',
            lower_bound = -1000,
            upper_bound = 1000,
        )

uptake_methane = cobra.Reaction(
            'uptake_methane',
            name = 'Methane Uptake from Environent',
            lower_bound = 0,
            upper_bound = 1.0
        )

methane_oxidation = cobra.Reaction(
            'methane_oxidation',
            name = 'methane oxidation',
            lower_bound = 0.0,
            upper_bound = 1000.0
    )

# add involved metabolites and stoichiometry

EX_methane.add_metabolites(
    {
        b_methane: -1.0,
    }
)

uptake_methane.add_metabolites(
    {
        b_methane: -1.0,
        e_methane: 1.0
    }
)

# pMMO reaction without redox coenzyme (in literature mostly ubiquinol is mentioned but neither is there
# cytosolic ubiqunol in this model (only in mitochondria), nor does the literature agree on what its role may be exactly )

methane_oxidation.add_metabolites(
    {
        e_methane: -1.0,
        pheast.metabolites.m1232: -1.0,
        pheast.metabolites.m1215: 1.0,
        pheast.metabolites.m139: 1.0,
    }
)

# add gene dependency for pMMO

methane_oxidation.gene_reaction_rule = 'AOX_pMMO'

# add reactions to pheast

pheast.add_reactions([EX_methane,uptake_methane,methane_oxidation])

Ignoring reaction 'EX_methane' since it already exists.
Ignoring reaction 'uptake_methane' since it already exists.
Ignoring reaction 'methane_oxidation' since it already exists.


In [187]:
pheast.reactions.r1158.bounds = 0, 0  # set methanol uptake to 0 and try if it grows on methane

In [188]:
# it grows on methane

pheast.optimize()
pheast.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


In [252]:
# we will now introduce our heterologous protein: leghemoglobin
# we look at the detoxification pathway as there is a heme group in both catalase and hemoglobin which could
# influence our system in general and reaction we introduce for production of (leg)hemoglobin

pheast.metabolites.query("H2O2","name")

[<Metabolite m118 at 0x7fb6a648d880>,
 <Metabolite m139 at 0x7fb6a648d460>,
 <Metabolite m140 at 0x7fb6a648d430>,
 <Metabolite m713 at 0x7fb6a65b26a0>,
 <Metabolite m1179 at 0x7fb6a65daee0>]

In [230]:
# this is the peroxisomal H2O2

pheast.metabolites.m713

Metabolite identifier,m713
Name,H2O2_H2O2
Memory address,0x07fb6a65b26a0
Formula,
Compartment,C_p
In 16 reaction(s),"r225, r91, r216, r222, r221, r220, r217, r99, r226, r224, r223, r218, r1088, r134, r219, r215"


In [254]:
# this is the catalase reaction, heme is not considered

pheast.reactions.r99

Reaction identifier,r99
Name,hydrogen-peroxide:hydrogen-peroxide oxidoreductase
Memory address,0x07fb6a5c0fdf0
Stoichiometry,2.0 m713 --> 2.0 m65 2.0 H2O2_H2O2 --> 2.0 H2O_H2O
GPR,PAS_chr2-2_0131
Lower bound,0.0
Upper bound,1000.0


In [260]:
# we find there is a heme metabolite

pheast.metabolites.query("heme","name")

[<Metabolite m1045 at 0x7fb6a65f85e0>,
 <Metabolite m1046 at 0x7fb6a65f8610>,
 <Metabolite m1047 at 0x7fb6a65f8640>,
 <Metabolite m1048 at 0x7fb6a65f8670>,
 <Metabolite m1049 at 0x7fb6a65f86a0>,
 <Metabolite m1055 at 0x7fb6a65f87c0>,
 <Metabolite m1056 at 0x7fb6a65f87f0>,
 <Metabolite m1185 at 0x7fb6a65da040>]

In [289]:
# it is siroheme

pheast.metabolites.m1056

Metabolite identifier,m1056
Name,Coproporphyrinogen I_C36H44N4O8
Memory address,0x07fb6a65f87f0
Formula,
Compartment,C_c
In 1 reaction(s),r860


In [258]:
# only involved in this reaction; it is also specifically a species of heme different from the one in catalse
# and hemoglobin, so we should not take this one
pheast.reactions.r973

Reaction identifier,r973
Name,S-Adenosyl-L-methionine:uroporphyrin-III C-methyltransferase
Memory address,0x07fb6a5e67a60
Stoichiometry,m1052 + m564 --> m1053 + 2.0 m16 Sirohydrochlorin_C42H46N4O16 + Fe2+_Fe --> Siroheme_C42H44FeN4O16 + 2.0 H+_H
GPR,PAS_chr1-4_0222
Lower bound,0.0
Upper bound,1000.0


In [279]:
# there are also a bunch of porypherin metabolites which are similar to the hemoglobin

pheast.metabolites.query("porphyrin","name")

#     A. Díaz, P.C. Loewen, I. Fita, X. Carpena
#    Thirty years of heme catalases structural biology
#    Arch. Biochem. Biophys., 525 (2012), pp. 102-110

# According to the source above C34-heme b is the most abundant, so we could go for that (there are some C34
# poryphyrins) and introduce it in the catalase and later hemoglobin reaction but none are in the peroxisome

[<Metabolite m1045 at 0x7fb6a65f85e0>,
 <Metabolite m1046 at 0x7fb6a65f8610>,
 <Metabolite m1047 at 0x7fb6a65f8640>,
 <Metabolite m1048 at 0x7fb6a65f8670>,
 <Metabolite m1049 at 0x7fb6a65f86a0>,
 <Metabolite m1055 at 0x7fb6a65f87c0>,
 <Metabolite m1056 at 0x7fb6a65f87f0>,
 <Metabolite m1185 at 0x7fb6a65da040>]

In [ ]:
# now we will introduce the heterologous proteins, pMMO and leghemoglobin, with reactions for dna replication,
# transcription and translation
# as the sequences are long, we calculate the stoichiometry with a script which is on the github repo and
# based on the logic behind introduction of heterologous protein production in the paper:
# Irani ZA, Kerkhoven EJ, Shojaosadati SA, Nielsen J (2016) Genome-scale metabolic model of Pichia pastoris with native and humanized glycosylation of recombinant proteins. Biotechnol Bioeng 113:961–969. doi: 10.1002/bit.25863.

# first remove the heterologous protein production reactions from the paper

#...

In [ ]:
# make new reactions

pMMO_DNA = cobra.Reaction(
            'pMMO_DNA',
            name = 'pMMO DNA replication',
            lower_bound = 0.0,
            upper_bound = 1000,
        )

pMMO_RNA = cobra.Reaction(
            'pMMO_RNA',
            name = 'pMMO transcription',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

pMMO_AA = cobra.Reaction(
            'pMMO_AA',
            name = 'pMMO translation',
            lower_bound = 0.0,
            upper_bound = 1000.0
    )

hemo_DNA = cobra.Reaction(
            'hemo_DNA',
            name = 'hemoglobin DNA replication',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_RNA = cobra.Reaction(
            'hemo_RNA',
            name = 'hemoglobin transcription',
            lower_bound = 0.0,
            upper_bound = 1000.0
        )

hemo_AA = cobra.Reaction(
            'hemo_AA',
            name = 'hemoglobin translation',
            lower_bound = 0.0,
            upper_bound = 1000.0
)


# add involved metabolites and stoichiometry

pMMO_DNA.add_metabolites(
    {
        ### deoxynucleotides
        # A
        pheast.metabolites.m404: -0.97513,
        # T
        pheast.metabolites.m437: -0.97513,
        # C
        pheast.metabolites.m431: -0.64352,
        # G
        pheast.metabolites.m389: -0.64352,
        # ATP
        pheast.metabolite.m1: -11.00682,
    }
)

pMMO_RNA.add_metabolites(
    {
        ### RNA nucleotides
        # A
        pheast.metabolites.m94: -1.11694,
        # U
        pheast.metabolites.m418: -0.72523,
        # C
        pheast.metabolites.m423: -0.74,
        # G
        pheast.metabolites.m384: -0.53415,
        # ATP
        pheast.metabolite.m1: -7.47917,
    }
)

pMMO_AA.add_metabolites(
    {
        ### Amino Acids
        # A
        pheast.metabolites.m153: -0.73616,
        # C
        pheast.metabolites.m331: -0.02974,
        # H
        pheast.metabolites.m490: -0.19333,
        # M
        pheast.metabolites.m343: -0.29,
        # T
        pheast.metabolites.m305: -0.59487,
        # R
        pheast.metabolites.m158: -0.38667,
        # E
        pheast.metabolites.m154: -0.43872,
        # I
        pheast.metabolites.m239: -0.54282,
        # F
        pheast.metabolites.m272: -0.53539,
        # W
        pheast.metabolites.m280: -0.29744,
        # N
        pheast.metabolites.m161: -0.28256,
        # Q
        pheast.metabolites.m163: -0.1859,
        # L
        pheast.metabolites.m227: -0.84769,
        # P
        pheast.metabolites.m185: -0.43872,
        # Y
        pheast.metabolites.m274: -0.40154,
        # D
        pheast.metabolites.m156: -0.42385,
        # G
        pheast.metabolites.m210: -0.69898,
        # K
        pheast.metabolites.m203: -0.3941,
        # S
        pheast.metabolites.m279: -0.43872,
        # V
        pheast.metabolites.m222: -0.70641,
        # ATP
        pheast.metabolite.m1: -38.11352,
    }
)

hemo_DNA.add_metabolites(
    {
        ### deoxynucleotides
        # A
        pheast.metabolites.m404: -1.0424,
        # T
        pheast.metabolites.m437: -1.0424,
        # C
        pheast.metabolites.m431: -0.57636,
        # G
        pheast.metabolites.m389: -0.57636,
        # ATP
        pheast.metabolite.m1: -11.00757,
    }
)

hemo_RNA.add_metabolites(
    {
        ### RNA nucleotides
        # A
        pheast.metabolites.m94: -1.2408,
        # U
        pheast.metabolites.m418: -0.80329,
        # C
        pheast.metabolites.m423: -0.6455,
        # G
        pheast.metabolites.m384: -0.43034,
        # ATP
        pheast.metabolite.m1: -7.48783,
    }
)

hemo_AA.add_metabolites(
    {
        ### Amino Acids
        # A
        pheast.metabolites.m153: -1.48373,
        # C, no Cystein in leghemoglobin
        # pheast.metabolites.m331:,
        # H
        pheast.metabolites.m490: -0.12902,
        # M
        pheast.metabolites.m343: -0.06451,
        # T
        pheast.metabolites.m305: -0.38706,
        # R
        pheast.metabolites.m158: -0.06451,
        # E
        pheast.metabolites.m154: -0.6451,
        # I
        pheast.metabolites.m239: -0.38706,
        # F
        pheast.metabolites.m272: -0.58059,
        # W
        pheast.metabolites.m280: -0.12902,
        # N
        pheast.metabolites.m161: -0.25804,
        # Q
        pheast.metabolites.m163: -0.32255,
        # L
        pheast.metabolites.m227: -0.83863,
        # P
        pheast.metabolites.m185: -0.32255,
        # Y
        pheast.metabolites.m274: -0.19353,
        # D
        pheast.metabolites.m156: -0.51608,
        # G
        pheast.metabolites.m210: -0.51608,
        # K
        pheast.metabolites.m203: -0.90314,
        # S
        pheast.metabolites.m279: -0.83863,
        # V
        pheast.metabolites.m222: -0.77412,
        # ATP
        pheast.metabolite.m1: -40.22199,
        # 
    }
)


# add reactions to pheast

pheast.add_reactions([pMMO_DNA,
                     pMMO_RNA,
                     pMMO_AA,
                     hemo_DNA,
                     hemo_RNA,
                     hemo_AA])

In [291]:
pheast.reactions.query("DNA","name")

[<Reaction r640 at 0x7fb6a5fabca0>,
 <Reaction r641 at 0x7fb6a5fabc70>,
 <Reaction r642 at 0x7fb6a5fab9d0>,
 <Reaction r649 at 0x7fb6a5fabb80>,
 <Reaction r650 at 0x7fb6a5fa3ca0>,
 <Reaction r651 at 0x7fb6a5fa3790>,
 <Reaction r652 at 0x7fb6a5fa3c10>,
 <Reaction r643 at 0x7fb6a5f6ebe0>,
 <Reaction r644 at 0x7fb6a5f6e970>,
 <Reaction r653 at 0x7fb6a5f7c730>,
 <Reaction r654 at 0x7fb6a5f7c490>,
 <Reaction r655 at 0x7fb6a5f6e9a0>,
 <Reaction r656 at 0x7fb6a5f6eac0>,
 <Reaction r1204 at 0x7fb6a5bd0d30>,
 <Reaction r1092 at 0x7fb6a5b47b80>]

In [294]:
pheast.reactions.r1101

Reaction identifier,r1101
Name,"FAB, polymerization of deoxynucleotides"
Memory address,0x07fb6a5b47370
Stoichiometry,10.447 m1 + 0.569 m389 + 1.182 m404 + 0.584 m431 + 0.737 m437 + 10.447 m5 --> m1362 + 10.447 m3 + 10.447 m7 10.447 ATP_C10H16N5O13P3 + 0.569 dGMP_C10H14N5O7P + 1.182 dAMP_C10H14N5O6P + 0.584 dCMP_C9H14N3O7P + 0.737 dTMP_C10H15N2O8P + 10.447 H2O_H2O --> FABdna_ + 10.447 ADP_C10H15N5O10P2 + 10.447...
GPR,
Lower bound,0.0
Upper bound,1000.0


In [297]:
pheast.metabolites.m3

Metabolite identifier,m7
Name,Orthophosphate_H3PO4
Memory address,0x07fb6a6489190
Formula,
Compartment,C_c
In 114 reaction(s),"r146, r638, r700, r473, r706, r701, r468, r705, r1101, r1303, r953, r857, r1058, r703, r469, r164, r1304, r713, r438, r1103, r467, r1053, r736, r725, r1074, r735, r1324, r1100, r727, r844, r145,..."


In [26]:
pheast.compartments

{'C_c': 'cytosol',
 'C_e': 'extracellular',
 'C_m': 'mitochondrion',
 'C_p': 'peroxisome',
 'C_v': 'vacuole',
 'C_g': 'golgi',
 'C_r': 'ER',
 'C_b': 'boundary'}

In [63]:
method_list = [method for method in dir(pheast) if method.startswith('__') is False]
print(method_list)

['_annotation', '_compartments', '_contexts', '_id', '_populate_solver', '_repr_html_', '_sbml', '_set_id_with_model', '_solver', '_tolerance', '_trimmed', '_trimmed_genes', '_trimmed_reactions', 'add_boundary', 'add_cons_vars', 'add_groups', 'add_metabolites', 'add_reaction', 'add_reactions', 'annotation', 'boundary', 'compartments', 'constraints', 'copy', 'demands', 'description', 'exchanges', 'genes', 'get_associated_groups', 'get_metabolite_compartments', 'groups', 'id', 'medium', 'merge', 'metabolites', 'name', 'notes', 'objective', 'objective_direction', 'optimize', 'problem', 'reactions', 'remove_cons_vars', 'remove_groups', 'remove_metabolites', 'remove_reactions', 'repair', 'sinks', 'slim_optimize', 'solver', 'summary', 'tolerance', 'variables']


In [87]:
dir(pheast)

['__add__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotation',
 '_compartments',
 '_contexts',
 '_id',
 '_populate_solver',
 '_repr_html_',
 '_sbml',
 '_set_id_with_model',
 '_solver',
 '_tolerance',
 '_trimmed',
 '_trimmed_genes',
 '_trimmed_reactions',
 'add_boundary',
 'add_cons_vars',
 'add_groups',
 'add_metabolites',
 'add_reaction',
 'add_reactions',
 'annotation',
 'boundary',
 'compartments',
 'constraints',
 'copy',
 'demands',
 'description',
 'exchanges',
 'genes',
 'get_associated_groups',
 'get_metabolite_compartments',
 'groups',
 'id',
 'medium',
 'merge',
 'metabolite